# 1.Скачиваем датасет и подготавливаем данные для применения uplift модели

In [ ]:
!wget 'https://drive.google.com/uc?id=1a47_w6J2G2k_IQNCaPX5JO56-4qfM1pJ' -O data.csv

# оно же у меня здесь
# with open('/content/drive/MyDrive/ГБ/выборки для исследований/HW_data.csv', "rb") as f:
#     df = pd.read_csv(f)


--2022-11-01 17:56:00--  https://drive.google.com/uc?id=1a47_w6J2G2k_IQNCaPX5JO56-4qfM1pJ
Resolving drive.google.com (drive.google.com)... 142.251.2.102, 142.251.2.113, 142.251.2.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jf7ss9l65ev8kslsv9nkiunpcoajgcnu/1667325300000/14904333240138417226/*/1a47_w6J2G2k_IQNCaPX5JO56-4qfM1pJ?uuid=eb472cc1-11e0-47a3-85e4-0266a59a7e75 [following]
--2022-11-01 17:56:00--  https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jf7ss9l65ev8kslsv9nkiunpcoajgcnu/1667325300000/14904333240138417226/*/1a47_w6J2G2k_IQNCaPX5JO56-4qfM1pJ?uuid=eb472cc1-11e0-47a3-85e4-0266a59a7e75
Resolving doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.googleusercontent.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to doc-00-c0-

In [ ]:
!pip install scikit-uplift catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier 

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold 

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import f1_score, roc_auc_score, log_loss, precision_recall_curve

from sklift.models import TwoModels, ClassTransformation
from sklift.metrics import uplift_at_k

%matplotlib inline

pd.set_option("display.max_rows", 6)

In [ ]:
df = pd.read_csv('data.csv', sep=',')
data =  df.copy()

data.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [ ]:
data.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
data.treatment= np.where((data.treatment == 'No Offer'), 0, 1) 
data

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
...,...,...,...,...,...,...,...,...,...
63997,6,29.99,1,0,Urban,1,Phone,1,0
63998,1,552.94,1,0,Surburban,1,Multichannel,1,0
63999,1,472.82,0,1,Surburban,0,Web,1,0


In [ ]:
y = data.target
X = data.drop(columns='target')

In [ ]:
X.history = MinMaxScaler().fit_transform(X[['history']])
X = pd.get_dummies(X, columns=['zip_code', 'channel'])
X.head(3)

,recency,history,used_discount,used_bogo,is_referral,treatment,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,10,0.033912,1,0,0,1,0,1,0,0,1,0
1,6,0.090198,1,1,1,0,1,0,0,0,0,1
2,7,0.045435,0,1,1,1,0,1,0,0,0,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# формируем переменные с флагом взаимодействия (индексы сохраняются)
treat_train = X_train['treatment']
treat_test = X_test['treatment']

X_train.drop(columns='treatment', inplace=True)
X_test.drop(columns='treatment', inplace=True)

In [ ]:
treat_test.value_counts()

1    12700
0     6500
Name: treatment, dtype: int64

# 2. Применяю трансформирующую модель аплифта на весь дата-тест

In [ ]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=1, silent=True))
ct = ct.fit(X_train, y_train, treat_train)

uplift_pred = ct.predict(X_test)
X_test['uplift_pred'] = uplift_pred

# Выведем топ 6 клиентов для взаимодействия
X_test.sort_values(by='uplift_pred', ascending = False).head(6)

,recency,history,used_discount,used_bogo,is_referral,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,uplift_pred
3993,12,0.097173,1,0,0,0,0,1,0,0,1,0.294932
47307,12,0.098443,1,0,0,0,0,1,0,0,1,0.294932
36314,12,0.055287,1,1,0,0,0,1,1,0,0,0.155996
17422,12,0.098310,1,1,0,0,0,1,0,0,1,0.137781
21114,12,0.098847,1,1,0,0,0,1,0,0,1,0.137781
36159,12,0.063578,1,1,0,1,0,0,1,0,0,0.121273


# 3. Далее разобъем этот дата-тест на 10 частей-блоков с посчитанным аплифт скором и сравним усредненные значения метрики успешности будущего взаимодействия

__Разбиваем на блоки:__

In [ ]:
X_test['container'] = pd.qcut(X_test.index, q=10, labels=False) #  из-за того что поделил так qcut(X_test['uplift_pred'] получился блок где тритмент с одним значением - только 1
X_test['container'].value_counts()

2    1920
9    1920
7    1920
     ... 
5    1920
4    1920
6    1920
Name: container, Length: 10, dtype: int64

__Считаем:__

In [ ]:
container_results = {
    'container': [],
    'uplift@10%': [],
    'uplift@20%': [],
    'uplift@30%': []
}

def compute_cont_result(uplift_pred):
  for i in sorted(X_test['container'].unique()): # перебираем секции, на которые разделили тестовые данные (10 секций)
    container_results['container'].append(i)
    y_test_indx = X_test.loc[X_test.container == i, :].index # индексы, соответствующие индексам данных выбранной секции
    for n in [0.1, 0.2, 0.3]:
      sm_score = uplift_at_k(y_true=y_test[y_test_indx], uplift=uplift_pred[y_test_indx], treatment=treat_test[y_test_indx], strategy='by_group', k=n)
      container_results[f'uplift@{int(n * 100)}%'].append(sm_score)

compute_cont_result(X_test.uplift_pred) # поскольку X_test уже не участвует в подсчете скора, можно передать в кач-ве переменной тритмент столбец uplift_pred в X_test

pd.set_option("display.max_rows", 12)
pd.DataFrame(container_results)

,container,uplift@10%,uplift@20%,uplift@30%
0,0,0.027778,0.021961,0.038135
1,1,0.150646,0.111685,0.092090
2,2,0.103593,0.067279,0.080107
3,3,0.130828,0.124200,0.125974
4,4,-0.037156,0.017966,0.051474
5,5,0.119218,0.096441,0.057865
6,6,-0.002630,0.037360,0.031817
7,7,0.088224,0.092839,0.068168
8,8,0.046762,0.041798,0.060971
9,9,0.079636,0.087136,0.076794


__Могу показать бизнесу по каждому блоку список наиболее целесообразных ко взаимодействию пользователей__

__В реальном примере будет сгенерирован весь список, а здесь в условиях компактности покажем например топ 10 у 7-го блока__

In [ ]:
X_test.loc[X_test.container == 7, :].sort_values(by='uplift_pred', ascending = False).head(10)

,recency,history,used_discount,used_bogo,is_referral,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,uplift_pred,container
47307,12,0.098443,1,0,0,0,0,1,0,0,1,0.294932,7
44852,12,0.062664,1,1,0,1,0,0,1,0,0,0.103586,7
49325,12,0.220975,1,1,1,0,0,1,1,0,0,0.074106,7
50963,1,0.182986,1,1,1,1,0,0,0,1,0,0.071280,7
45239,1,0.146520,1,1,1,1,0,0,0,1,0,0.067071,7
48857,1,0.174982,1,1,1,1,0,0,0,1,0,0.067071,7
48081,2,0.180860,1,1,1,1,0,0,0,1,0,0.060023,7
49580,10,0.108150,1,0,0,0,0,1,0,0,1,0.052177,7
48712,10,0.224877,1,1,1,0,1,0,0,1,0,0.045405,7
46951,10,0.281305,1,1,1,0,1,0,0,1,0,0.045405,7


__Сравниваем аплифт скора с разбиением на блоки и без разбиения__

In [ ]:
models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': [],
    'uplift@30%': []
}

def compute_result(model_name, uplift_model):
  models_results['approach'].append(model_name)
  for i in [0.1, 0.2, 0.3]:
    sm_score = uplift_at_k(y_true=y_test, uplift=uplift_model, treatment=treat_test, strategy='by_group', k=i)
    models_results[f'uplift@{int(i * 100)}%'].append(sm_score)

ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=1, silent=True))
ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_test)


compute_result('ClassTransform', uplift_ct)

In [ ]:
pd.DataFrame(models_results)

,approach,uplift@10%,uplift@20%,uplift@30%
0,ClassTransform,0.07358,0.068934,0.066481


Сравним с усредненными показателями ранних расчетов по всем 10 блокам

In [ ]:
pd.DataFrame(container_results)['uplift@10%'].mean(), pd.DataFrame(container_results)['uplift@20%'].mean(), pd.DataFrame(container_results)['uplift@30%'].mean()

(0.07068984731557701, 0.06986668095139406, 0.06833941813651936)

Отличия есть, но незначительные



# 4. Реализуем эту же идею, но чтобы модель аплифта работала с каждым блоком отдельно, выше же аплифт применен на весь дата-тест.

__это самый корректный и эффективный подход__

In [ ]:
pd.set_option("display.max_rows", 4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

indices_train = X_train.index 
indices_test = X_test.index

treat_train = X_train.loc[indices_train, 'treatment']
treat_test = X_test.loc[indices_test, 'treatment']

X_train.drop(columns='treatment', inplace=True)
X_test.drop(columns='treatment', inplace=True)

In [ ]:
# разбиваю на блоки так, чтобы потом можно было соотнести по индексам наименование блока к строке в тестовых данных
X_test['container'] = pd.qcut(X_test.index, q=10, labels=False)
containers = X_test['container']

# !!!!!!! видим что строка 24540 есть - внизу будет проблемка
X_test.loc[X_test.container == 3, :]


,recency,history,used_discount,used_bogo,is_referral,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web,container
24540,2,0.234326,1,0,1,0,1,0,0,0,1,3
23038,10,0.010941,0,1,1,0,1,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
23364,10,0.005648,0,1,1,1,0,0,0,1,0,3
24161,1,0.086766,0,1,0,0,1,0,0,0,1,3


In [ ]:
# подчищаю X_test, чтобы трансформер обучился корректно (без доп. признака)
X_test.drop(columns='container', inplace=True)

In [ ]:
pd.set_option("display.max_rows", 12)

containers = pd.DataFrame(containers)
containers.value_counts()

container
0            1920
1            1920
2            1920
3            1920
4            1920
5            1920
6            1920
7            1920
8            1920
9            1920
dtype: int64

In [ ]:
X_test.isnull().sum()

recency                 0
history                 0
used_discount           0
used_bogo               0
is_referral             0
zip_code_Rural          0
zip_code_Surburban      0
zip_code_Urban          0
channel_Multichannel    0
channel_Phone           0
channel_Web             0
dtype: int64

Видим что по всем 10 блокам сформировано по 1920 строк, в блоках пустых срок нет

In [ ]:
pd.set_option("display.max_rows", 6)

# !!!!!!! видим что строка 24540 есть как и выше мы видели что в X_test эта строка тоже есть
containers.loc[containers.container == 3, :]

,container
24540,3
23038,3
24734,3
...,...
21303,3
23364,3
24161,3


In [ ]:
for i in range(10):
  print(treat_test.iloc[containers.loc[containers.container == 2, :].index].value_counts())
  print('*'*20)

1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************
1    1282
0     638
Name: treatment, dtype: int64
********************


В каждом блоке есть флаг взаимодействия и отсутствия взаимодействия, т.е. оба значение (0, 1)

__Далее в цикле обрубаю работу алгоритма на 3 блоке, т.к. в ходе цикла получал ошибку:__

IndexError: positional indexers are out-of-bounds

__Опытным путем (в том цикле печатал этапы по отработке блоков)  увидел что на 3 обрубается, потому в след. ячейке пробовал вывести индекс строки в тестовых данных, относящейся к третьему блоку и получаю такую же ошибку__

In [ ]:
# X_test.iloc[24540]

#-------------
#out
# IndexError: single positional indexer is out-of-bounds

In [ ]:
container_results = {
    'container': [],
    'uplift@10%': [],
    'uplift@20%': [],
    'uplift@30%': []
}

def compute_cont_result():
  for i in sorted(containers['container'].unique()): # перебираем секции, на которые разделили тестовые данные (10 секций)
    if i == 3: # почему-то в третьем блоке как-будто нет данных(либо куда-то делись индексы)
      break
    container_results['container'].append(i)
    y_test_indx = containers.loc[containers.container == i, :].index # индексы, соответствующие индексам данных выбранной секции

    ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=1, silent=True))
    ct = ct.fit(X_train, y_train, treat_train)
    print(f'Отработан блок {i}')
    uplift_ct = ct.predict(X_test.iloc[y_test_indx]) # предсказываю только в рамках данных одного блока

    for n in [0.1, 0.2, 0.3]:
      sm_score = uplift_at_k(y_true=y_test[y_test_indx], uplift=uplift_ct, treatment=treat_test[y_test_indx], strategy='by_group', k=n)
      container_results[f'uplift@{int(n * 100)}%'].append(sm_score)
    

      

compute_cont_result() # поскольку X_test уже не участвует в подсчете скора, можно передать в кач-ве переменной тритмент столбец uplift_pred в X_test



pd.set_option("display.max_rows", 12)
pd.DataFrame(container_results)

Отработан блок 0
Отработан блок 1
Отработан блок 2


,container,uplift@10%,uplift@20%,uplift@30%
0,0,-0.002728,0.033926,0.022592
1,1,-0.011956,0.047386,0.035926
2,2,0.095226,0.082782,0.071874
